# Algebrajski tipi

Poleg bogatega nabora vgrajenih tipov si tipe v OCamlu lahko definiramo tudi sami.

## Okrajšave tipov

Najenostavnejši način za definicijo tipov so okrajšave obstoječih tipov. Na primer, tip za $\mathbb{R}^3$ si lahko definiramo kot:

In [ ]:
type r3 = float * float * float

Tako kot na vgrajena tipa `list` in `option` lahko tudi naši tipi vsebujejo parametre:

In [ ]:
type 'a zaporedje = int -> 'a

Če tip sprejme več parametrov (na primer slovar ima tako tip ključev kot tip vrednosti), jih lahko naštejemo v oklepajih.

In [ ]:
type ('k, 'v) slovar = ('k * 'v) list

### Označba tipov

Tudi če si definiramo svoj tip, bo OCaml privzeto izračunal najbolj splošni tip:

In [ ]:
let vsota_r3 (x1, y1, z1) (x2, y2, z2) =
  (x1 +. x2, y1 +. y2, z1 +. z2)

Če želimo vsiliti svoj tip, to na poljubnem vzorcu ali izrazu storimo z označbo `(... : tip)`. Tip rezultata funkcije vsilimo z `let ... : tip = ...`.

In [ ]:
let vsota_r3 ((x1, y1, z1) : r3) ((x2, y2, z2) : r3) : r3 =
  (x1 +. x2, y1 +. y2, z1 +. z2)

Alternativni način zgornje označbe je tudi:

In [ ]:
let vsota_r3' : r3 -> r3 -> r3 =
 fun (x1, y1, z1) (x2, y2, z2) -> (x1 +. x2, y1 +. y2, z1 +. z2)

## Zapisni tipi

Kompleksna števila predstavimo s pari realnih števil:

In [ ]:
type kompleksno = float * float


Kako bi izračunali absolutno vrednost kompleksnega števila? Ena možnost je:

In [ ]:
let abs (x, y) = sqrt (x ** 2. +. y ** 2.)

Toda če smo v mislih imeli polarni zapis, je pravilna definicija:

In [ ]:
let abs (r, _) = r

Ali pa datume, ki jih ponavadi predstavimo s trojico celih števil:

In [ ]:
type datum = int * int * int

Kateri vrstni red smo uporabili: dan, mesec, leto, kot smo navajeni v Sloveniji, ali leto, mesec, dan, kot je mednarodni standard? Mogoče celo mesec, dan, leto, kot je navada v Združenih državah?

### Definicija zapisnih tipov


Zmešnjavi se lahko izognemo, če komponente poimenujemo. V OCamlu to storimo z zapisnimi tipi, ki jih podamo tako, da naštejemo imena polj ter njihove tipe:

In [ ]:
type kartezicno = {re : float; im : float}
type polarno = {radij : float; kot : float}
type datum = { dan : int; mesec : int; leto : int }

Vrednosti tipov pišemo podobno, le da jih podamo z `=`:

In [ ]:
let i = {re = 0.; im = 1.}
let i' = { radij = 1.; kot = 0.}
let osamosvojitev = { dan = 25; mesec = 6; leto = 1991 }

Kljub temu, da zapise pišemo podobno kot Pythonove slovarje, gre za popolnoma različni strukturi. Zapisi so poimenovani kartezični produkti, torej heterogeni, Pythonovi slovarji pa so običajno homogeni, torej so vsi ključi enega in vse vrednosti drugega tipa. Poleg tega imena polj zapisov niso vrednosti, ki bi si jih lahko podajali naokoli.

### Razstavljanje zapisov

Do posameznih komponent dostopamo z `zapis.ime_polja`:

In [ ]:
let abs z = sqrt (z.re ** 2. +. z.im ** 2.)

In [ ]:
let abs' z = z.radij

In [ ]:
let je_prestopno leto =
  (leto mod 4 = 0 && leto mod 100 <> 0) || leto mod 400 = 0
  
let dolzina_meseca leto =
  function
  | 4 | 6 | 9 | 11 -> 30
  | 2 -> if je_prestopno leto then 29 else 28
  | _ -> 31

In [ ]:
let je_veljaven datum =
  let veljaven_dan = 1 <= datum.dan && datum.dan <= dolzina_meseca datum.leto datum.mesec
  and veljaven_mesec = 1 <= datum.mesec && datum.mesec <= 12
  in
  veljaven_dan && veljaven_mesec

Včasih je krajše, če zapise razstavimo s pomočjo vzorcev oblike `{ polje1 = vzorec1; polje2 = vzorec2; … }`:

In [ ]:
let je_veljaven {dan = d; mesec = m; leto = l} =
  let veljaven_dan = 1 <= d && d <= dolzina_meseca l m
  and veljaven_mesec = 1 <= m && m <= 12
  in
  veljaven_dan && veljaven_mesec

Če za vzorce uporabimo spremenljivke z enakimi imeni kot polja, lahko uporabimo tudi krajšo obliko:

In [ ]:
let je_veljaven {dan = dan; mesec = mesec; leto = leto} =
  let veljaven_dan = 1 <= dan && dan <= dolzina_meseca leto mesec
  and veljaven_mesec = 1 <= mesec && mesec <= 12
  in
  veljaven_dan && veljaven_mesec

In [ ]:
let je_veljaven {dan; mesec; leto} =
  let veljaven_dan = 1 <= dan && dan <= dolzina_meseca leto mesec
  and veljaven_mesec = 1 <= mesec && mesec <= 12
  in
  veljaven_dan && veljaven_mesec

In [ ]:
let abs {re; im} = sqrt (re ** 2. +. im ** 2.)

Če želimo v vzorcu kakšna polja izpustiti, jih nadomestimo z podčrtajem:

In [ ]:
let abs' { radij; _ } = radij

In [ ]:
let je_pred_nasim_stetjem {leto; _} =
  leto <= 0

### Posodabljanje zapisov

Z zapisom `{zapis with polje1 = vrednost1; …}` ustvarimo nov zapis, ki ima z izjemo naštetih vrednosti polja enaka prvotnemu:

In [ ]:
let konjugiraj z = {z with im = -. z.im}

In [ ]:
let pred_sto_leti datum =
  {dan = datum.dan; mesec = datum.mesec; leto = datum.leto - 100}

In [ ]:
let pred_sto_leti datum =
  {datum with leto = datum.leto - 100}

### Pametni konstruktorji

Z lastnimi tipi lahko dosežemo tudi strožje preverjanje veljavnosti vrednosti. Napišimo funkcijo `naredi_datum`, ki vrne datum le, če dani trije argumenti predstavljajo veljaven datum. Takim funkcijam pravimo _pametni konstruktorji_, saj konstrukcijo vrednosti obogatijo z dodatno logiko.

In [ ]:
let naredi_datum dan mesec leto =
  let datum = { dan; mesec; leto } in
  if je_veljaven datum then Some datum else None

In [ ]:

  naredi_datum 29 2 2000


In [ ]:

  naredi_datum 29 2 1900


Če smo previdni in za konstrukcijo datumov uporabimo le to funkcijo, bodo vsi datumi v našem programu veljavni. Kmalu bomo videli, da nam tudi previden ne bo treba biti, saj bomo uporabo pametnih konstruktorjev tudi vsilili.

## Naštevni tipi

Najzanimivejši tipi, ki jih lahko definiramo, so _naštevni tipi_. Tako kot pri zapisnih tipih bomo tudi vrednosti naštevnih tipov sestavljali iz manjših vrednosti. Razlika med njimi je v tem, da morajo biti pri zapisnih tipih prisotne vrednosti _vseh_ naštetih polj, mora biti pri naštevnih tipih prisotna _natanko ena_ izmed naštetih variant.

Recimo, da si želimo definirati tip pošiljk iz spletne trgovine. Zaenkrat imena in naslove predstavimo z nizi. Tudi vrsto dostave (osebno, po pošti) bi lahko predstavili z nizi.

In [ ]:
type posiljka = {
  naslovnik : string;
  naslov : string;
  dostava : string
}

In [ ]:
[
  { naslovnik = "Matija Pretnar"; naslov = "5.19"; dostava = "osebni prevzem"};
  { naslovnik = "Katja Berčič"; naslov = "5.07"; dostava = "po pošti"};
  { naslovnik = "Filip Koprivec"; naslov = "P.23"; dostava = "osebno"};
]

Vidimo, da lahko v nize zapišemo karkoli, kar otežuje obdelavo podatkov, pa tudi vodi v nesmisle. Ena možnost je, da dostavo opišemo z logično vrednostjo:

In [ ]:
type posiljka = {
  naslovnik : string;
  naslov : string;
  osebni_prevzem : bool
}

In [ ]:
[
  { naslovnik = "Matija Pretnar"; naslov = "5.19"; osebni_prevzem = true };
  { naslovnik = "Katja Berčič"; naslov = "5.07"; osebni_prevzem = false };
  { naslovnik = "Filip Koprivec"; naslov = "P.23"; osebni_prevzem = true };
]

Ta rešitev ni idealna, saj ne vemo, kaj vse implicira vrednost `false`. Še več: kaj, če bi želeli dodati še tretjo možnost, na primer hitro dostavo? Boljši način je, da uporabimo naštevne tipe.

### Definicije naštevnih tipov

Naštevne tipe podamo tako, da naštejemo možne variante, od katerih je vsaka podana s svojim _konstruktorjem_.

In [ ]:
type dostava =
  | OsebniPrevzem
  | PoPosti

type posiljka = {
  naslovnik : string;
  naslov : string;
  dostava : dostava
}

Tedaj bo tip imel natanko dve možni vrednosti in OCaml nas bo opozoril, če poskusimo uporabiti nenavedeno varianto:


In [ ]:
[
  { naslovnik = "Matija Pretnar"; naslov = "5.19"; dostava = OsebniPrevzem };
  { naslovnik = "Katja Berčič"; naslov = "5.07"; dostava = PoPosti };
  { naslovnik = "Filip Koprivec"; naslov = "P.23"; dostava = Osebno };
]

### Razstavljanje naštevnih tipov

Tako kot naštevne tipe podamo po kosih, lahko prek `match` ali `function` po kosih tudi definiramo funkcije na njih.

In [ ]:
type dostava =
  | OsebniPrevzem
  | PoPosti

In [ ]:
let cena_dostave =
  function
  | OsebniPrevzem -> 0.
  | PoPosti -> 2.5

Če tip razširimo z dodatno varianto, nas bo prevajalnik sam opozoril nanjo:

In [ ]:
type dostava =
  | OsebniPrevzem
  | PoPosti
  | HitraDostava

In [ ]:
let cena_dostave =
  function
  | OsebniPrevzem -> 0.
  | PoPosti -> 2.5

In [ ]:
let cena_dostave =
  function
  | OsebniPrevzem -> 0.
  | PoPosti -> 2.5
  | HitraDostava -> 4.

### Konstruktorji z argumenti

Vsak izmed naštetih konstruktorjev lahko sprejme tudi argumente vnaprej določenega tipa:

In [ ]:
type dostava =
  | OsebniPrevzem
  | PoPosti of string
  | HitraDostava of string
type posiljka = {
  naslovnik : string;
  dostava : dostava
}

In [ ]:
HitraDostava "Jadranska ulica 21, 1000 Ljubljana"

In [ ]:
type geometrijski_objekt =
  | Tocka
  | Krog of float
  | Pravokotnik of float * float

In [ ]:
let ploscina =
  function
  | Tocka -> 0.
  | Krog r -> 3.14 *. r ** 2.
  | Pravokotnik (v, s) -> v *. s

In [ ]:
[Tocka; Pravokotnik (1., 2.); Tocka; Krog 3.]
|> List.map ploscina
|> List.fold_left (+.) 0.

Primer naštevnega tipa, ki ga že poznamo, je tip `option`:

In [ ]:
type 'a option = None | Some of 'a

### Naštevni tipi z eno samo varianto

Naštevni tipi so lahko koristni tudi takrat, ko imajo samo eno varianto, saj s tem lahko ločimo vrednosti, ki so predstavljene z istim osnovnim tipom. Recimo, tako naslove kot telefonske številke lahko predstavimo z nizi. Vendar če za dve vrsti vrednosti uporabljamo isti tip, lahko to vodi do zmešnjave, kakršna se je [zgodila mestecu Gold Hill](https://mathwithbaddrawings.com/2015/08/19/the-smartest-dumb-error-in-the-great-state-of-colorado/).

Če pa definiramo dva različna naštevna tipa, bo na to namesto nas pazil prevajalnik:

In [ ]:
type naslov = Naslov of string
type telefon = Telefon of string

type dostava =
  | OsebniPrevzem
  | PoPosti of naslov
  | HitraDostava of naslov * telefon

type posiljka = {
  naslovnik : string;
  dostava : dostava
}

In [ ]:
{ naslovnik = "Matija Pretnar";
  dostava = HitraDostava (
    Telefon "01 4766 600",
    Naslov "Jadranska 21"
  )}

## Algebrajski tipi

Naštevni tipi so lahko tudi **rekurzivni**. Takim tipom pravimo algebrajski, v nekaterih primerih, ki jih bomo spoznali kasneje, pa tudi induktivni.

### Naravna števila

Najenostavnejši primer induktivnega tipa so naravna števila. Predstavimo jih z naštevnim tipom s konstruktorjema `Nic` in `Naslednik`, pri čemer slednji sprejme en argument, ki je zopet naravno število.

In [ ]:
type naravno =
  | Nic
  | Naslednik of naravno

Vsoto naravnih števil podamo z običajno rekurzivno definicijo:

In [ ]:
let rec vsota m n =
  match m with
  | Nic -> n
  | Naslednik m' -> Naslednik (vsota m' n)

In [ ]:
let ena = Naslednik Nic
let dva = vsota ena ena
let stiri = vsota dva dva
let sest = vsota stiri dva

In [ ]:
vsota sest sest

### Verižni seznami

Še en znan primer induktivnega tipa so verižni seznami. Vsak seznam je bodisi prazen, bodisi sestavljen iz glave in repa:

In [ ]:
type 'a seznam =
  | Prazen
  | Sestavljen of 'a * 'a seznam

Sedaj tudi vidimo, zakaj `::` lahko uporabljamo v vzorcih - ni namreč običajna funkcija za sestavljanje seznamov, temveč konstruktor tipa seznamov.

### Aritmetični izrazi

Induktivne tipe se pogosto uporablja za predstavitev izrazov določenega formalnega jezika. Na primer, aritmetične izraze gradimo iz števil ter aritmetičnih operacij. Take izraze bi lahko predstavili s tipom:

In [ ]:
type izraz =
  | Stevilo of int
  | Plus of izraz * izraz
  | Minus of izraz
  | Krat of izraz * izraz

Na primer, izrazu $-(5 \times (2 + 7))$ bi ustrezala vrednost

In [ ]:
let i = Minus (
  Krat (Stevilo 5, Plus (Stevilo 2, Stevilo 7))
)

Za vajo lahko napišete rekurzivno funkcijo `izracunaj : izraz -> int`, ki dani izraz prevori v njegovo vrednost. Na primer, za zgornji izraz bi funkcija vrnila `-45`.

### Dvojiška drevesa

Še en induktivni tip, ki ga bomo podrobneje spoznali v kratkem, pa so dvojiška drevesa. Dvojiško drevo je bodisi prazno bodisi ima koren, v katerem je shranjena vrednost, ter dva otroka, ki sta zopet drevesi, na primer (pri čemer praznih dreves ne kažemo):

![](slike/09-iskalna-drevesa/avl-drevo.png)

Tip dvojiških dreves podamo s tipom

In [ ]:
type 'a drevo =
  | Prazno
  | Sestavljeno of 'a drevo * 'a * 'a drevo


## Vaje

### Kompleksna števila